# Summary
- Introduce Gluster, formulation, and Gluster objective (details of Gluster algo after experiments)
- Random Features experiments. Conclusion: There is a limitted range for both Gluster and SVRG to work.
- Which datasets are in that regime?
- Positives: extra duplicates added to NQM, MNIST, CIFAR-10, CIFAR-100
- Negatives: Total distortions plot as a function of # clusters. CIFAR10-Imagenet-NLP
- Details of Gluster algo and implementation

## Plots

* RF variance vs overparametrization (mostly ready)
* MNIST train/var plots
* CIFAR10 var plots and plus noise
* NQM, MNIST, CIFAR-10, CIFAR100 +duplicate var plots
* CIFAR-10,Imagenet,NLP(bow/bert) Total distortion plots (notice total distortions has Nk multiplier explain and compare to random)

# Experiments Left
- CIFAR10 raw data variance plot
- Imagenet duplicates var plot (online)
- NLP vs C plots
- normalized var plots

# Emails

Summary:
- Results on Noisy Quadratic Model (linear regression with noise in the labels) are in favor of Gluster.
- The issue on CIFAR10 where the Gluster estimator had high variance can be resolved by adding noise to the data. It seems to be a data problem that doesn't appear as strongly on CIFAR-100 and Imagenet. But very specific types of noise can fix that.
- Gluster shines with extra duplicates added to NQM, MNIST, CIFAR-10, CIFAR-100. The variance is lower than SGD with double the mini-batch size.
- On Imagenet, the gradients do not form clusters, hence no improvement on uniform sampling. For this part, I have a new plot that measures the performance of clustering and how it compares to uniform sampling.
- I have the sketch of a proof that for a fixed memory footprint, our rank-1 formulation achieves the lowest clustering objective possible compared to any rank-k formulation.

Note that:
- With the exception of NQM, I did not use Gluster for training. I measured the variance of Gluster on the trajectory of mini-batch SGD with the same mini-batch size.
- Gluster is said to perform well and shine when its variance is lower than SGD with twice the mini-batch size.
- In all experiments I used batch Gluster, i.e., I fix the model a few times during the training, I do a number of full assignment/update steps and update the sampling. This version of Gluster is slower but more accurate with fewer hyperparameters.

On Wed, Mar 25, 2020 at 12:30 PM David Duvenaud <duvenaud@cs.toronto.edu> wrote:
Nice!  A few comments:

1) It seems a bit funny to compare to SGD with twice the minibatch size.  Why not just compare with the same minibatch sizes?

Good point. That is one of our 3 baselines we compare to:
1) SGD with the same mini-batch size: We want to at least beat this. With the exception of CIFAR-10 (without noise) Gluster is always as good or better than this.
2) SGD with double the mini-batch size: Gluster has extra memory foot-print for every cluster center equal to the memory taken by the mini-batch. If we are better than 2xB SGD, Gluster is worth the extra memory.
3) SVRG: SVRG is only good for training on simple linear regression and MNIST. It also has extra computation cost. But since it is a variance reduction method, it is a natural baseline. On NQM we get very close to SVRG with much less computation cost.

2) Maybe another nice demo would be for unbalanced data.  If our method automatically up-weighted the rare classes, that would be another selling point.

That's an excellent point.
Would it be more interesting if I downsample a few classes from MNIST/CIFAR-10?
Or should I create a toy problem similar to NQM but with multiple classes?

3) It seems strange that adding noise to the data reduced the variance of our gradient estimator.  Do you have an idea why that could have helped?  I am worried there might be a bug.

This is the problem I was stuck at for a year. Let me clarify the problem and my findings.
Quick correction: adding noise increases variance as you expected.
But I'm not comparing the absolute value of the variance with and without the noise.
I comparing Gluster to SGD without noise and then separately Gluster to SGD with noise.

Problem:
With Gluster, we don't have any guarantee that clustering stays optimal when the model changes during training.
In theory, a non-smooth objective function can have very different gradients after a single optimization step.
We can constructs examples where Gluster becomes worse than uniform sampling.
We have always hoped that this is not true, and the clustering stays valid for a while.
Hopefully, longer than the time it takes the control variate of SVRG to become stale.

Problem manifestation:
On CIFAR-10 however, gradients can change rapidly. See the attached variance plot cifar10_resnet8_smoothing.png (this is with label smoothing, without label smoothing is a bit more spiky).
For half the training, the variance of Gluster estimator is lower than SGD but after the learning rate drop, spikes appear.
It is very important to observe that the high variance points are spikes. The gradients suddenly become completely different but then they change back again.
Geometrically, it can mean the model is oscillating between different basins of attraction. I'm not sure about that.
My hypothesis is that, CIFAR-10 is such that the model can overfit to it, but not fully. Maybe because of mislabeled or ambiguous data. But something makes the model change a lot.

Solution (or rather making data favourable to Gluster)
Again, it is important that the problem is spikes, not consistently high variance as in SVRG. So maybe we can solve it.
I thought let's treat the problem as having some sort of non-smoothness and try to smoothen it.
So I tried adding different types of noise. In particular, corrupting a portion of labels (changing them to something random) got rid of the spikes (cifar10_resnet8_corrupt.png)

With label corruption, I'm changing the task and data but as long as there is not too much corruption, a trained model will generalize to the true data.

On when optimal sampling could help: https://arxiv.org/pdf/1412.0156.pdf
On when the bias and variance of deep learning models: https://arxiv.org/pdf/2002.11328.pdf

These papers have conflicting observations. So I think it might be instructive to run a set of experiments to figure out which of the following regime: overparameterized vs under parameterized vs right at the interpolation threshold, our gluster / optimal sampling data points will work better.

I think we should go back to our linear regression and multinomial regression tasks, instead of working with the linear models, we need to run experiments with random kitchen sinks and only learns the top layer weights. The label will also be generated by a teacher random kitchen sink. 

The point of this exercise is that we can now vary the number of hidden units, d, in a random kitchen sink to simulate the three regimes: overparameterized (d >>n) vs under parameterized (n >>d) vs right at the interpolation threshold (n ~= d). 

So far we believe that when d >>n, non of the variance reduction methods matter, we verified this hypothesis on linear models but that may not be the same as neural networks, a better study is to look at the random kitchen sink experiments. I suspect that gluster should work well for the under parameterized (n >>d) regime.

Thanks David for the reminder. I was at his talk and discussed Gluster with him in a meeting.
I think their results based on the SGC assumption are closely related. If we our hypothesis holds in RF experiments we can build a theory based on SGC. He was also interested.
His slides for that talk:
https://www.cs.ubc.ca/~schmidtm/Documents/2020_Vector_SmallResidual.pdf

I'm working on the random features/kitchen sink experiments. We should have more definite answers for the underparametrized regime there.

By the way, I also tried imbalance data on both CIFAR-10 and MNIST.
Again good variance reduction with Gluster on MNIST but fluctuating and high variance on CIFAR-10.
This doesn't affect our hypothesis about parametrization.
We will be able to do robust optimization in settings like MNIST where Gluster is stable and good.

Please find plots attached for random features models in underparam/overparam regimes. Details are below.
Please let me know your thoughts.

Conclusions:
- It looks like we need more overparametrization for Gluster to work but too much overpamaterization leaves no room for variance reduction.
- Gluster seem to be as good as SGD-2B with mild overparametrization (1-4).
- Gluster and SVRG are both unstable with large learning rate in underparametrized regime (<=1).
- In highly overparametrized regime (10), the gain from SVRG vanishes.
- Here are rough overparametrization coefficients I had been working with (not accounting for data augmentation):
  + MNIST CNN: 37, MLP: 31  (consistent variance reduction below SGD-2B)
  + CIFAR-10 Resnet8: 3, Resnet32: 9 (unstable variance reduction)
  + ImageNet ResNet18: 10 (consistently no variance reduction)
- If we account for data augmentation, assuming each training data gives us at least 10 new data points, all models I tried on CIFAR-10 and ImageNet are underparametrized.

More observations:
- The only hyperparameters that affect the variance are learning rate and the ratio of student_hidden/num_train_data. Each dot is an average over all other hyperparameters. Small error bars show that other hyperpameters do not matter.
- Contrary to our expectation, the ratio student_hidden/teacher_hidden does not affect the variance of the gradient.
- Gluster is always between SGD-B (same mini-batch size) and SGD-2B (double mini-batch size). Almost never worse than SGD-B, never better than SGD-2B.
- Gluster has high variance with large learning rate (0.1) in the underparametrized regime and the interpolation point.
- The gain of SVRG vanishes in the over-parametrized regime in 2 ways: 1) it provides less variance reduction 2) all methods have relatively low variance.
- One bonus plot: the variance of SGD for 3 learning rates together in plot shows that the variance of the gradient is smaller for larger learning rates and the gap grows as overparametrization grows.

Experimental setting:
- The random features (RF) model is a 2 layer binary classification model where the first layer weights are fixed. Only the second layer weights are trained. The first layer's activation is Relu and the model is trained with cross-entropy loss. Each random feature is sampled from a normal distribution and normalized to L2 norm 1.
- Data is generated from a Gaussian and labeled by a teacher RF model.
- We train a student RF model on this data.
- Important hyperparameters:
  + dim: Dimensionality of input
  + teacher_hidden
  + student_hidden
  + num_train_data
  + learning rate

How to read plots:
- There are 3 plots for 3 learning rates (0.1, 0.01, 0.001)
- The y-axis (in log-scale) is the mean/max variance over the last 70% of the training (this is to ignore the first epoch where SVRG and Gluster do not have a good variance estimate). Max plot should capture fluctuations of a variance estimator more.
- The x-axis is the over-parametrization coefficient (student_hidden/num_train_data). Each point is generated by keeping student_hidden fixed (1000) and varying num_train_data (in the range [0.1, 10]).
- We average over different values of the rest of hyperpameters:
  + Multiple random seeds (3)
  + teacher_hidden (0.1x and 10x student hidden)
  + input dim (0.1x and 10x student_hidden)
  + momentum=0
  + weight decay=1e-4